# FEMA's National Flood Insurance Policy Database Data Exploration

### Load Libraries

In [21]:
import pandas as pd
import matplotlib.pyplot as plt

from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Imputer
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, ByteType, LongType, FloatType, ShortType
from pyspark.sql.functions import col, sum as spark_sum

### Initialize Spark Session

In [22]:
spark = SparkSession.builder \
    .config("spark.driver.memory", "10g") \
	.config("spark.executor.memory", "2g") \
    .config('spark.executor.instances', 5) \
	.config("spark.sql.debug.maxToStringFields", "100")\
    .appName("Flood Data") \
	.getOrCreate()

ConnectionRefusedError: [Errno 111] Connection refused

### Load data to Spark dataframe; Infer Schema

In [3]:
#path to data 
data_path = "./NFIP/nfip-flood-policies.csv"

df = spark.read.option("header","true").option("inferSchema","true").csv(data_path)

## Perform the Data Exploration
Below is the outline of the Data Exploration Section:
- Print Schema
- Column Description
- Display the Number of Variables (Columnns)
- Show the First Few Rows
- Display Number of Observations
- Missing Values in the Dataframe
- Statistics Summary and Data Distribution
- Correlations Among Variables
- Scatterplots

### Print Schema

Understanding the structure of data (column names and types) is crucial.

In [4]:
df.printSchema()

root
 |-- agriculturestructureindicator: string (nullable = true)
 |-- basefloodelevation: double (nullable = true)
 |-- basementenclosurecrawlspacetype: integer (nullable = true)
 |-- cancellationdateoffloodpolicy: date (nullable = true)
 |-- censustract: long (nullable = true)
 |-- condominiumindicator: string (nullable = true)
 |-- construction: string (nullable = true)
 |-- countycode: integer (nullable = true)
 |-- crsdiscount: double (nullable = true)
 |-- deductibleamountinbuildingcoverage: string (nullable = true)
 |-- deductibleamountincontentscoverage: string (nullable = true)
 |-- elevatedbuildingindicator: string (nullable = true)
 |-- elevationcertificateindicator: string (nullable = true)
 |-- elevationdifference: integer (nullable = true)
 |-- federalpolicyfee: integer (nullable = true)
 |-- floodzone: string (nullable = true)
 |-- hfiaasurcharge: integer (nullable = true)
 |-- houseofworshipindicator: string (nullable = true)
 |-- latitude: double (nullable = true)
 |--

### Column Description

**Description of the columns from the FEMA's National Flood Insurance Policy Database, grouped by their data types and purpose:**

**Geographic and Location Details**
- **censustract (long):** Census tract number indicating the specific area where the property is located, used for demographic analysis.
- **countycode (integer):** Numeric code representing the county in which the property is insured.
- **floodzone (string):** Designation of the flood zone according to FEMA's mapping, crucial for assessing the property's flood risk.
- **latitude (double), longitude (double):** Geographic coordinates specifying the precise location of the property.
- **propertystate (string):** The U.S. state where the property is located.
- **reportedcity (string):** The city reported for the insured property.
- **reportedzipcode (integer):** Zip code where the property is situated, used for localizing insurance coverage and risk.

**Property and Construction Details**
- **agriculturestructureindicator (string):** Indicates whether the property is used for agricultural purposes.
- **basementenclosurecrawlspacetype (integer):** Type of basement or crawlspace present at the property, affecting flood risk assessment.
- **construction (string):** Describes the type of construction materials and methods used, which can affect the property's vulnerability to flood damage.
- **numberoffloorsininsuredbuilding (integer):** Total floors in the insured building, important for determining potential flood damage and insurance coverage needs.
- **elevatedbuildingindicator (string):** Indicates whether the building is elevated, a key factor in reducing flood risk.

**Policy Details**
- **policycost (integer):** The total cost of the flood insurance policy.
- **policycount (integer):** The number of policies associated with a single property or account.
- **policyeffectivedate (date), policyterminationdate (date):** Start and end dates of the flood insurance coverage.
- **totalbuildinginsurancecoverage (integer), totalcontentsinsurancecoverage (integer):** The amount of insurance coverage for the building and its contents, respectively.
- **totalinsurancepremiumofthepolicy (integer):** Total premium amount for the flood insurance policy.

**Flood Risk Assessment Specifics**
- **basefloodelevation (double):** The base flood elevation expected for a particular area, critical for understanding flood risk levels.
- **elevationcertificateindicator (string), elevationdifference (integer):** Presence of an elevation certificate and the difference in elevation, respectively, both crucial for assessing compliance with floodplain management.
- **lowestadjacentgrade (double), lowestfloorelevation (double):** Measures of elevation that help determine the property's flood exposure.

**Insurance Policy Features**
- **crsdiscount (double):** Community Rating System discount applied to the policy, which can reduce insurance premiums based on community flood preparedness.
- **deductibleamountinbuildingcoverage (integer), deductibleamountincontentscoverage (integer):** Deductible amounts for building and contents coverage, influencing out-of-pocket costs after a flood.
- **hfiaasurcharge (integer):** Surcharge applied under the Homeowner Flood Insurance Affordability Act.
- **federalpolicyfee (integer):** A fee associated with the federal policy governing flood insurance.

**Special Indicators**
- **condominiumindicator (string), primaryresidenceindicator (string):** Indicate whether the insured property is a condominium or the primary residence of the owner.
- **houseofworshipindicator (string), nonprofitindicator (string):** Indicators of whether the property is used as a house of worship or is owned by a nonprofit organization, affecting policy terms and possibly qualifying for special considerations.
- **postfirmconstructionindicator (string):** Indicates if the building was constructed after the community's first Flood Insurance Rate Map was issued, which can affect insurance rates.
- **smallbusinessindicatorbuilding (string):** Indicates whether the insured building is used for small business purposes.

**Additional Policy and Coverage Information**
- **originalconstructiondate (date), originalnbdate (date):** Dates of original construction and the building's initial notebook entry, important for historical property assessments.
- **cancellationdateoffloodpolicy (date):** Date when the flood policy was cancelled, if applicable.
- **regularemergencyprogramindicator (string):** Indicates the type of FEMA program under which the policy is covered, distinguishing between regular and emergency management programs.
- **ratemethod (integer):** Describes the method used to calculate the insurance rate, impacting how premiums are determined.
- **locationofcontents (string):** Specifies where within the property the insured contents are located, relevant for claims and risk assessments.

### Display number of Variables (Columns)


In [5]:
num_variables = len(df.columns)
print("Number of Columns:", num_variables)

Number of Columns: 45


### Show the First Few Rows

In [6]:
df.show(5,vertical=True)

-RECORD 0--------------------------------------------------
 agriculturestructureindicator      | NULL                 
 basefloodelevation                 | NULL                 
 basementenclosurecrawlspacetype    | 2                    
 cancellationdateoffloodpolicy      | NULL                 
 censustract                        | 33013038500          
 condominiumindicator               | N                    
 construction                       | N                    
 countycode                         | 33013                
 crsdiscount                        | 0.0                  
 deductibleamountinbuildingcoverage | 0                    
 deductibleamountincontentscoverage | 0                    
 elevatedbuildingindicator          | N                    
 elevationcertificateindicator      | NULL                 
 elevationdifference                | 999                  
 federalpolicyfee                   | 13                   
 floodzone                          | X 

### Display Number of Observations

In [7]:
df.count()

50406943

FEMA's National Flood Insurance Policy Database, containing over 50 million (50,406,943) policy transactions.

### Missing Values in the Dataframe

In [8]:
#find number of missing values in the dataframe
missing_vals = df.select(*(spark_sum(col(i).isNull().cast("int")).alias(i) for i in df.columns))

In [9]:
missing_vals.show(vertical= True)

-RECORD 0--------------------------------------
 agriculturestructureindicator      | 38923313 
 basefloodelevation                 | 33636759 
 basementenclosurecrawlspacetype    | 802      
 cancellationdateoffloodpolicy      | 43614057 
 censustract                        | 467119   
 condominiumindicator               | 6        
 construction                       | 13       
 countycode                         | 48999    
 crsdiscount                        | 0        
 deductibleamountinbuildingcoverage | 661993   
 deductibleamountincontentscoverage | 5561584  
 elevatedbuildingindicator          | 258      
 elevationcertificateindicator      | 32606397 
 elevationdifference                | 0        
 federalpolicyfee                   | 0        
 floodzone                          | 169145   
 hfiaasurcharge                     | 0        
 houseofworshipindicator            | 34476251 
 latitude                           | 338699   
 locationofcontents                 | 15

**Here's a description of the missing value situation in the FEMA Flood Insurance Policy Database:**

1. **High Missing Values:**
-  **Cancellation Date of Flood Policy, Obstruction Type:** These fields each have 43,614,057 and 40,629,070 missing values.
-  **Agriculture Structure Indicator, Non-profit Indicator, House of Worship Indicator, Base Flood Elevation:** These fields have between 38,923,313 and 33,636,759 missing values.
-  **Small Business Indicator Building, Lowest Floor Elevation, Elevation Certificate Indicator:** These fields have between 33,451,148 and 32,606,397 missing values.

2. **Moderate Missing Values:**
- **Deductible Amount in Building Coverage, Deductible Amount in Contents Coverage:** Missing 15,649,149 and 18,265,104 values respectively, which implies incomplete data on policy deductibles that could affect premium calculations and risk assessments.
- **Location of Contents:** With 15,389,767 missing entries, there's substantial missing information on where contents are located within the insured buildings, which is vital for damage assessments.

3. **Low Missing Values:**
- **Census Tract, Flood Zone:** Missing 467,119 and 169,145 entries respectively. Although relatively lower, these still represent significant gaps, especially as these fields are crucial for location-specific risk assessment.
- **Number of Floors in Insured Building:** Missing data on 162,301 entries could affect understanding building structure and associated risk.

4. **Minimal to No Missing Values:**
- Fields like **CRS Discount, Federal Policy Fee**, and various policy-related dates (effectiveness, termination) and costs show zero missing values, indicating complete data in terms of policy transaction details.
- Similarly, **County Code, Construction, Condominium Indicator, Occupancy Type** show minimal missing data (under 50,000), suggesting good coverage of basic property and policyholder information.

Overall, the dataset shows a strong presence of policy and basic property information. However, gaps in data can hamper effective risk assessment and pricing of flood insurance policies, especially in areas prone to flooding where such data is most critical. Addressing these missing values, either by data imputation where appropriate or by collecting missing data could significantly enhance the robustness of any analysis or predictive modeling based on this dataset.

### Statistics Summary and Data Distribution

In [10]:
numerical_columns = [col_name for col_name, data_type in df.dtypes if data_type in ["int", "bigint", "double"]]

# Select only numerical columns
numerical_df = df.select(*numerical_columns)

# Generate summary statistics
summary_stats = numerical_df.describe()

# Show summary statistics
summary_stats.show(vertical = True)

-RECORD 0------------------------------------------------
 summary                          | count                
 basefloodelevation               | 16770184             
 basementenclosurecrawlspacetype  | 50406141             
 censustract                      | 49939824             
 countycode                       | 50357944             
 crsdiscount                      | 50406943             
 elevationdifference              | 50406943             
 federalpolicyfee                 | 50406943             
 hfiaasurcharge                   | 50406943             
 latitude                         | 50068244             
 longitude                        | 50068244             
 lowestadjacentgrade              | 15466364             
 lowestfloorelevation             | 17346341             
 numberoffloorsininsuredbuilding  | 50244642             
 occupancytype                    | 50406937             
 policycost                       | 50406943             
 policycount  

The summary statistics for the FEMA National Flood Insurance Policy Database provide a comprehensive overview of various policy and property-related numerical attributes. These statistics include measures of central tendency, dispersion, and range, all of which are critical for understanding the distribution and potential data quality issues within the dataset. Below is a detailed analysis of the key statistical summaries:

**Central Tendency and Dispersion**
1. **Base Flood Elevation:**
- Average (Mean): 119.47 ft.
- Standard Deviation: 522.49 ft.
- Range: -9999 - 85,640 ft.
2. **Lowest Adjacent Grade:**
- Average (Mean): 129.20 ft.
- Standard Deviation: 609.92 ft.
- Range: -9,999 - 99,990.9 ft.
3. **Lowest Floor Elevation:**
- Average (Mean): 385.62 ft.
- Standard Deviation: 1,676.42 ft.
- Range: -9,997.9 - 99,989 ft.
4. **Basement Enclosure Crawl Space Type:**
- Average (Mean): 0.37, indicating a slight bias towards lower classifications.
- Standard Deviation: 0.86, showing moderate variability within the data.
- Range: Min 0 to Max 4, spanning several classification levels.
5. **Census Tract:**
- Average (Mean): Approximately 2.6 x 10¹⁰.
- Standard Deviation: About 1.58 x 10¹⁰, suggesting a wide spread across census tracts.
6. **CRS Discount:**
- Average: 0.064, typically low across the dataset.
- Standard Deviation: 0.091, with most data points close to zero but some higher values.
7. **Deductible Amount in Building and Contents Coverage:**
- Building Coverage Average: 1.66 with a deviation of 1.46.
- Contents Coverage Average: 0.98 with a deviation of 1.05.
- Both show low average deductible amounts but with notable variation.
8. **Elevation Difference:**
- Average: 1.69, indicating minor differences in elevation on average.
- Standard Deviation: 3.39, suggesting significant outliers affecting the elevation difference.
9. **Policy Related Figures (Policy Cost, Policy Count, Total Insurance Coverage, etc.):**
- These values have a high mean and standard deviation, indicating a significant spread in the policy costs and coverages, reflecting diverse insurance policies and property valuations.

**Extremes (Minimum and Maximum Values)**
- Notable minimums include negative values in **Federal Policy Fee and HFIAA Surcharge**, possibly indicating refunds or adjustments.
- The maximum values in **Total Building Insurance Coverage and Total Insurance Premium of the Policy** reach into the hundreds of millions, highlighting cases with exceptionally high insurance coverage.

**Implications**
- The wide variability in policy costs and coverage levels underscores the diverse nature of the insured properties. Accurate and complete data in these fields are crucial for effective risk management and policy pricing in flood insurance.

- This analysis provides a basis for further data cleaning, particularly in addressing missing values and outliers, which are essential for improving data quality and the reliability of subsequent analyses and decision-making processes based on this dataset.

### Correlations Among Variables

In [11]:
numerical_df.show(5,vertical=True)

-RECORD 0---------------------------------------
 basefloodelevation               | NULL        
 basementenclosurecrawlspacetype  | 2           
 censustract                      | 33013038500 
 countycode                       | 33013       
 crsdiscount                      | 0.0         
 elevationdifference              | 999         
 federalpolicyfee                 | 13          
 hfiaasurcharge                   | 0           
 latitude                         | 43.3        
 longitude                        | -71.8       
 lowestadjacentgrade              | NULL        
 lowestfloorelevation             | NULL        
 numberoffloorsininsuredbuilding  | 2           
 occupancytype                    | 1           
 policycost                       | 388         
 policycount                      | 1           
 policytermindicator              | 1           
 totalbuildinginsurancecoverage   | 250000      
 totalcontentsinsurancecoverage   | 100000      
 totalinsurancepremi

In [12]:
num_df_dropped = numerical_df.drop("basefloodelevation","latitude","longitude","lowestadjacentgrade","lowestfloorelevation")

v_col = "features"
input = num_df_dropped.columns
output = [v_col + str(i) for i in range(len(input))]
imputer = Imputer(strategy="mean",inputCols=input,outputCols=output)
changed_df = imputer.fit(num_df_dropped).transform(num_df_dropped) 

assembler = VectorAssembler(inputCols=output,outputCol=v_col)
numerical_df_vector = assembler.transform(changed_df).select(v_col)
numerical_df_vector

DataFrame[features: vector]

In [13]:
matrix = Correlation.corr(numerical_df_vector,v_col).collect()[0][0]
corr_matrix = matrix.toArray().tolist()
print(corr_matrix)

[[1.0, 0.13623903586845487, 0.13222438896911154, -0.1337456840142641, 0.04144637455805757, 0.021515636223077703, 0.039205129679828934, 0.4205917981132781, -0.06389760287441464, 0.07213744564440354, 0.00879790297900046, -0.02450568557810686, 0.006946886998778841, -0.09892408517021709, 0.0693259281247627], [0.13623903586845487, 1.0, 0.9861052821687085, -0.23633772590850213, 0.14064291796967068, -0.04651095355139787, -0.0017065025678084957, 0.13092070392149222, -0.021518544770044803, -0.009406725795160381, -0.023542521100064738, 0.03562507918662197, -0.02664276057268567, 0.014045383330901513, -0.007888058597759437], [0.13222438896911154, 0.9861052821687085, 1.0, -0.23953380471436234, 0.13888862687868733, -0.04591061242264458, -0.0038094031436245192, 0.1266704346367964, -0.020833364315469626, -0.010202709301517732, -0.023350267827395927, 0.04136907348096722, -0.027360032357225787, 0.008361075386161203, -0.008511916163268995], [-0.1337456840142641, -0.23633772590850213, -0.23953380471436234

In [14]:
num_df_dropped.columns

['basementenclosurecrawlspacetype',
 'censustract',
 'countycode',
 'crsdiscount',
 'elevationdifference',
 'federalpolicyfee',
 'hfiaasurcharge',
 'numberoffloorsininsuredbuilding',
 'occupancytype',
 'policycost',
 'policycount',
 'policytermindicator',
 'totalbuildinginsurancecoverage',
 'totalcontentsinsurancecoverage',
 'totalinsurancepremiumofthepolicy']

In [15]:
columns = ["basementenclosurecrawlspacetype",
 "censustract",
 "countycode",
 "crsdiscount",
 "elevationdifference",
 "federalpolicyfee",
 "hfiaasurcharge",
 "numberoffloorsininsuredbuilding",
 "occupancytype",
 "policycost",
 "policycount",
 "policytermindicator",
 "totalbuildinginsurancecoverage",
 "totalcontentsinsurancecoverage",
 "totalinsurancepremiumofthepolicy"]
df_c = spark.createDataFrame(corr_matrix,columns)
df_c.select("basementenclosurecrawlspacetype",
 "censustract",
 "countycode").show(15)

+-------------------------------+--------------------+--------------------+
|basementenclosurecrawlspacetype|         censustract|          countycode|
+-------------------------------+--------------------+--------------------+
|                            1.0| 0.13623903586845487| 0.13222438896911154|
|            0.13623903586845487|                 1.0|  0.9861052821687085|
|            0.13222438896911154|  0.9861052821687085|                 1.0|
|            -0.1337456840142641|-0.23633772590850213|-0.23953380471436234|
|            0.04144637455805757| 0.14064291796967068| 0.13888862687868733|
|           0.021515636223077703|-0.04651095355139787|-0.04591061242264458|
|           0.039205129679828934|-0.00170650256780...|-0.00380940314362...|
|             0.4205917981132781| 0.13092070392149222|  0.1266704346367964|
|           -0.06389760287441464|-0.02151854477004...|-0.02083336431546...|
|            0.07213744564440354|-0.00940672579516...|-0.01020270930151...|
|           

In [16]:
df_c.select("crsdiscount",
 "elevationdifference",
 "federalpolicyfee").show(15)

+--------------------+--------------------+--------------------+
|         crsdiscount| elevationdifference|    federalpolicyfee|
+--------------------+--------------------+--------------------+
| -0.1337456840142641| 0.04144637455805757|0.021515636223077703|
|-0.23633772590850213| 0.14064291796967068|-0.04651095355139787|
|-0.23953380471436234| 0.13888862687868733|-0.04591061242264458|
|                 1.0| -0.5259678054689083| 0.12618095833291207|
| -0.5259678054689083|                 1.0|-0.12308129153346878|
| 0.12618095833291207|-0.12308129153346878|                 1.0|
|0.048954413784206216|-0.04971959541878701| 0.15998037490116276|
| -0.0697807615905636|-0.02959368105409...| 0.10729663798512859|
| 0.04701164748485698|-0.05068082979967...|  0.1423736707514655|
| 0.06739894183937775|6.468266631658919E-4|  0.3864721337330112|
|0.028147108771781164|-0.03065010725766519|  0.5481074038104508|
|-0.03970568654901237| 0.04104422570965059|-0.02839057493420...|
|0.025212129977885477|-0.

In [17]:
df_c.select("hfiaasurcharge",
 "numberoffloorsininsuredbuilding",
 "occupancytype").show(15)

+--------------------+-------------------------------+--------------------+
|      hfiaasurcharge|numberoffloorsininsuredbuilding|       occupancytype|
+--------------------+-------------------------------+--------------------+
|0.039205129679828934|             0.4205917981132781|-0.06389760287441464|
|-0.00170650256780...|            0.13092070392149222|-0.02151854477004...|
|-0.00380940314362...|             0.1266704346367964|-0.02083336431546...|
|0.048954413784206216|            -0.0697807615905636| 0.04701164748485698|
|-0.04971959541878701|           -0.02959368105409...|-0.05068082979967...|
| 0.15998037490116276|            0.10729663798512859|  0.1423736707514655|
|                 1.0|           0.056225567874606905|  0.3014557593894687|
|0.056225567874606905|                            1.0|0.030588790472823736|
|  0.3014557593894687|           0.030588790472823736|                 1.0|
|  0.1823519286217399|            0.08060603844556684| 0.25987507020571776|
|0.031949873

In [18]:
df_c.select("policycost",
 "policycount",
 "policytermindicator").show(15)

+--------------------+--------------------+--------------------+
|          policycost|         policycount| policytermindicator|
+--------------------+--------------------+--------------------+
| 0.07213744564440354| 0.00879790297900046|-0.02450568557810686|
|-0.00940672579516...|-0.02354252110006...| 0.03562507918662197|
|-0.01020270930151...|-0.02335026782739...| 0.04136907348096722|
| 0.06739894183937775|0.028147108771781164|-0.03970568654901237|
|6.468266631658919E-4|-0.03065010725766519| 0.04104422570965059|
|  0.3864721337330112|  0.5481074038104508|-0.02839057493420...|
|  0.1823519286217399|0.031949873197437695|-0.02571907189115...|
| 0.08060603844556684| 0.06980684688803301|-0.04448801970119...|
| 0.25987507020571776| 0.08065471349337365|-0.02093908622872...|
|                 1.0| 0.49588758667292615|-0.00977337988857...|
| 0.49588758667292615|                 1.0|-0.00282092454406...|
|-0.00977337988857...|-0.00282092454406...|                 1.0|
|  0.5147020510962004|  0

In [19]:
df_c.select("totalbuildinginsurancecoverage",
 "totalcontentsinsurancecoverage",
 "totalinsurancepremiumofthepolicy").show(15)

+------------------------------+------------------------------+--------------------------------+
|totalbuildinginsurancecoverage|totalcontentsinsurancecoverage|totalinsurancepremiumofthepolicy|
+------------------------------+------------------------------+--------------------------------+
|          0.006946886998778841|          -0.09892408517021709|              0.0693259281247627|
|          -0.02664276057268567|          0.014045383330901513|            -0.00788805859775...|
|          -0.02736003235722...|          0.008361075386161203|            -0.00851191616326...|
|          0.025212129977885477|           -0.1443948051727502|            0.058638703250568816|
|          -0.03866066385828042|           0.11507960281910588|            0.009922795564580986|
|           0.49635660866817927|          -0.09552873635534823|             0.33594882023229616|
|           0.03871098206689112|          0.014049042681092588|              0.1114843251081501|
|          0.05960004760437341

### Scatterplots 

In [20]:
changed_num_df = changed_df.sample(False, 0.1)
num_pdf = changed_num_df.toPandas()
print(num_pdf)

----------------------------------------ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving

Exception occurred during processing of request from ('127.0.0.1', 56858)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
plt.scatter(x=num_pdf["policycost"],y=num_pdf["totalinsurancepremiumofthepolicy"])
plt.xlabel("Policy Cost")
plt.ylabel("Total Insurance Premium of the Policy")          
plt.title("Policy Cost vs Total Insurance Premium of the Policy")
plt.show()

In [ ]:
plt.scatter(x=num_pdf["countycode"],y=num_pdf["censustract"])
plt.xlabel("County Code")
plt.ylabel("Census Tract") 
plt.title("County Code vs Census Tract")
plt.show()

In [ ]:
plt.scatter(x=num_pdf["policycount"],y=num_pdf["totalbuildinginsurancecoverage"])
plt.xlabel("Policy Count")
plt.ylabel("Total Building Insurance Coverage") 
plt.title("Policy Count vs Total Building Insurance Coverage")
plt.show()